In [ ]:
#   __QUADROTORDYNAMICS__
#  This file implements the dynamics of a simple ,
#  quadrotor micro aerial vehicle

In [ ]:
import numpy as np
from scipy.integrate import odeint

In [ ]:
class QuadrotorDynamics:
    
    def __init__(self, save_state=True, config={}):
        '''
        Quadrotor Dynamics Parameters
        ------------
        save_state: Boolean
            Decides whether the states of the  system should be saved
            and returned at the end
        '''
        self.config = {      # configuration of the drone
            # Define constants
            'gravity': 9.8,    # Earh gravity, m/s^2
            # Define Vehicle Parameters
            'mass': 1,    # Mass, kg
            'length': 0.2,    # Arm length, m
            # Cross-Inertia [Ixx, Iyy, Izz]
            'inertia': np.array([0.0053, 0.0053, 0.0086]),    # kg/m^2
            'thrustToDrag': 0.018,    # thrust to drag constant, m
        }
        
        self.save_state = save_state
        self._dt  = 0.005    # Simulation time step
        self.config.update(config)
        self.initialize_state()
        
    def initialize_state(self):
        self.state = {
            # Position [x, y, z] of the quadrotor in inertial frame
            'position': np.zeros(3),
            # Velocity [dx/dt, dy/dt, dz/dt] of the quadrotor in inertial frame
            'velocity': np.zeros(3),
            # Euler angles [phi, theta, psi]
            'orientation': np.zeros(3),
            # Angular velocity [p, q, r]
            'ang_velocity': np.zeros(3)
        }
        
    def motor_thrust(self, moments, total_thrust):
        '''
        Compute Motor Thrusts
        
        Parameters
        -----------
        moments  : numpy.array
            [Mp, Mq, Mr] moments
        total_thrust  : float
            The total thrust generated by all motors
          
        Returns
        --------
        numpy.array
            The thrust generated by each motor [T1, T2, T3, T4]
        '''
        [Mp, Mq, Mr] = moments
        thrust = np.zeros(4)
        # due to yaw moment, Mr and adding it two total thrust
        tmp1add = total_thrust + Mr/self.config['thrustToDrag']
        tmp1sub = total_thrust - Mr/self.config['thrustToDrag']
        # due to roll and pitch moment, Mp and Mr
        tmp2p = 2*Mp / self.config['length']
        tmp2q = 2*Mq / self.config['length']
        
        # thrust of 4 motors  # based on Motor Mixing Algorithm described below
        # two are rotating clockwise and two anti-clockwise
        thrust[0] = tmp1add - tmp2q
        thrust[1] = tmp1sub + tmp2p
        thrust[2] = tmp1add + tmp2q
        thrust[3] = tmp1sub - tmp2p
        
        return thrust / 4.0
    
    def dt_eulerangles_to_angular_velocity(self, dtEuler, EulerAngles):
        '''
        Euler angles derivative To angular velocities
        dtEuler = np.array([dphi/dt, dtheta/dt, dpsi/dt])
        ## relation between [phidot, thetadot, psidot] and [p, q, r]
        '''
        return np.dot(self.angular_rotation_matrix(EulerAngles), dtEuler)
    
    def acceleration(self, thrusts, EulerAngles):
        '''
        Compute the acceleration in inrtial reference frame
        thrust = np.array([Motor1, Motor2, Motor3, Motor4])
        '''
        force_z_body = np.sum(thrusts) / self.config['mass']    # force in upward direction = thrust/mass
        rotation_matrix = self.rotation_matrix(EulerAngles)
        # print rotation matrix
        force_body = np.array([0, 0, force_z_body])
        # converting from body to inertial frame and substracting gravity force in body frame
        return np.dot(rotation_matrix, force_body) - np.array([0 , 0, self.config['gravity']])
    
    def angular_acceleration(self, omega, thrust):
        '''
        compute the angular acceration in body frame
        omega = angular velocity : -np.array([p, q, r])
        '''
        [t1, t2, t3, t4] = thrust
        thrust_matrix = np.array([self.config['length'] * (t2 - t4),   # L = roll
                                 self.config['length'] * (t3 - t1),    # M = pitch
                                 self.config['thrustToDrag'] * (t1 - t2 + t3 - t4)   # N = yaw
                                 ])
        
        inverse_inertia = np.linalg.inv(self.inertia_matrix)
        p1 = np.dot(inverse_inertia, thrust_matrix)
        p2 = np.dot(inverse_inertia, omega)
        p3 = np.dot(self.inertia_matrix, omega)
        cross = np.cross(p2, p3)
        # returns [pdot, qdot, rdot]
        return p1 - cross
    
    def angular_velocity_to_dt_eulerangles(self, omega, EulerAngles):
        '''
        Angular Velocity To Euler Angle
        omega = angular_velocity :  -np.array([p, q, r])
        ## relation between [p, q, r] and [phidot, thetadot, psidot]
        '''
        rotation_matrix = np.linalg.inv(self.angular_rotation_matrix(EulerAngles))
        return np.dot(rotation_matrix, omega)
    
    def moments(self, ref_acc, angular_vel):
        '''
        Compute the moments
        
        Parameters
        ------------
        ref_acc : numpy.array
            The desired angular accelration that the system should achieve.
            This should be of form [dp/dt, dq/dt, dr/dt]
        angular_vel : numpy.array
            The current angular velocity of the system.
            This should be of form [p, q, r]
        
        Returns
        ---------
        numpy.array
        The desired moments of the system
        returns desired [L, M, N] values
        '''
        inverse_inertia = np.linalg.inv(self.inertia_matrix)
        p1 = np.dot(inverse_inertia, angular_vel)
        p2 = np.dot(self.inertia_matrix, angular_vel)
        cross = np.cross(p1, p2)
        value = ref_acc + cross
        return np.dot(self.inertia_matrix, value)
    
    def angular_rotation_matrix(self, EulerAngles):
        '''
        Rotation matrix for Angular Velocity(omega=[p,q,r]) <-> Euler Angle[phi,theta,psi] Conversion
        Use inverse of the matrix to convert from angular velocity to euler rates
        '''
        [phi, theta, psi] = EulerAngles
        cphi = np.cos(phi)
        ctheta = np.cos(theta)
        cpsi = np.cos(psi)
        sphi = np.sin(phi)
        stheta = np.sin(theta)
        spsi = np.sin(psi)
        RotMatAngV = np.array([ [1,     0,     -stheta],
                                [0,  cphi, ctheta*sphi],
                                [0, -spsi, ctheta*cphi]
                              ])
        return RotMatAngV
    
    def rotation_matrix(self, EulerAngles):
        '''
        Rotation matrix :
                transform one coordinate system to another
                order of rotation =>  phi, theta, psi
        '''
        [phi, theta, psi] = EulerAngles
        cphi = np.cos(phi)
        ctheta = np.cos(theta)
        cpsi = np.cos(psi)
        sphi = np.sin(phi)
        stheta = np.sin(theta)
        spsi = np.sin(psi)
        
        RotMat = np.array([[ctheta*cpsi, sphi*stheta*cpsi - cphi*spsi, stheta*spsi + cphi*stheta*cpsi],
                           [ctheta*spsi, cphi*cpsi + sphi*stheta*spsi,   cphi*stheta*sphi - sphi*cpsi],
                           [    -stheta,                  sphi*ctheta,                    cphi*ctheta]
                          ])
        return RotMat
    
    @property
    def inertia_matrix(self):
        '''
        returns inertial matrix
        '''
        return np.diag(self.config['inertia'])
    
    def update_state(self, piecewise_args):
        '''
        Run the  state update equations for self._dt  seconds
        Update the current state of the system. It runs the  model and
        update its state to sel._dt seconds
        
        Parameters
        -----------
        piecewise_args : array
            It contains the parameters that are needed to run each section
            of the flight. It is an array of tuples.
            [(ct1, da1, t1), (ct2, da2, t2),......., (ctn, dan, tn)]
            
            ct : float
                The collective thrust generated by all motors
            da : numpy.array
                The desired angular acceleration that the system should achieve.
                This should be of form [dp/dt, dq/dt, dr/dt]
            t : float
                Time for which this section should run and should be atleast twice 
                self._dt
        '''
        if self.save_state:
            overall_time = 0
            for section in piecewise_args:
                overall_time += section[2]
        
            overall_length = len(np.arange(0, overall_time, self._dt)) - (len(piecewise_args) - 1)
            # Allocate space for storing state of all sections
            final_state = np.zeros([overall_length + 100, 12])
        else:
            final_state = []
            
        # create variable to maintain state between integration steps
        self._euler_dot = np.zeros(3)    # store states [phidot, thetadot, psidot]
        index = 0
        
        for section in piecewise_args:
            (total_thrust, desired_angular_acc, t) = section
            if t < (2 * self._dt):
                continue
            
            ts = np.arange(0, t, self._dt)
            state = np.concatenate((self.state['position'], self.state['velocity'],
                                  self.state['orientation'], self.state['ang_velocity']))
            output = odeint(self._integrator, state, ts, args=(total_thrust, desired_angular_acc))
            output_length = len(output)
            # State update
            # adding last element list of output array as new state 
            [self.state['position'], self.state['velocity'],
            self.state['orientation'], self.state['ang_velocity']] = np.split(output[output_length-1], 4)
            
            if self.save_state:
                # Final state update
                final_state[index:(index + output_length)] = output
                index = index + output_length - 1
            
        return final_state
    
    def _integrator(self, state, t, total_thrust, desried_angular_acc):
        '''
        Callback function for scipy.integrator.odeint
        At this point scipy used to execute the forward integration
        
        Parameters
        -----------
        state : numpy.array
            System State:  [x, y, z, xdot, ydot, zdot, phi, theta, psi, p, q, r]
        t : float
            Time
        
        Returns
        --------
        numpy.array
            Rates of the input state:
            [xdot, ydot, zdot, xddot, yddot, zddot, phidot, thetadot, psidot, pdot, qdot, rdot]
        '''
        # Position interial frame [x, y, z]
        pos = state[:3]
        # Velocity inertial frame [xdot, ydot, zdot]
        velocity  = state[3:6]
        # Euler Angles [phi, theta, psi]
        euler = state[6:9]
        # Angular velocity [p, q, r]
        omega = state[9:12]
        
        # Derivative of euler angles [dphi/dt, dtheta/dt, dpsi/dt]
        euler_dot = self._euler_dot
        
        # omega = self.dt_eulerangles_to_angular_velocity(euler_dot, euler)
        moments = self.moments(desried_angular_acc, omega)
        thrusts = self.motor_thrust(moments, total_thrust)
        # Acceleration in inertial frame
        acc = self.acceleration(thrusts, euler)
        
        omega_dot = self.angular_acceleration(omega, thrusts)
        euler_dot = self.angular_velocity_to_dt_eulerangles(omega, euler)
        self._euler_dot = euler_dot
        # [velocity : acc : euler_dot : omega_dot]
        # print(f'Vel = {velocity}')
        # print(f'Acc = {acc}')
        # print(f'Euler dot = {euler_dot}')
        # print(f'omega dot = {omega_dot}')
        return np.concatenate((velocity, acc, euler_dot, omega_dot))


In [ ]:
## drone geometry

################## TOP -VIEW OF THE DRONE

##                1(ac)         2(c) 
##                     #      # 
##                       ####
##                       ####
##                       ####
##                     #     # 
##                4(c)         3(ac) 

## c - clockwise rotation     ac - anti-clockwise rotation

In [ ]:
## Motor Mixing Algorithm
## --------------------------

# Motor_FR = Tcmd + Ycmd + Pcmd + Rcmd
# Motor_FL = Tcmd - Ycmd + Pcmd - Rcmd
# Motor_BR = Tcmd - Ycmd - Pcmd + Rcmd
# Motor_BL = Tcmd + Ycmd - Pcmd - Rcmd

# FR = Front Right motor   # FL = Front Left motor
# BR = Back Right motor    # BL = Back Left motor
# Tcmd = Thrust commanded
# Ycmd = Yaw commanded
# Pcmd = Pitch commanded
# Rcmd = Roll commanded